In [3]:
import cv2
import numpy as np
import os 
import tkinter as tk
from tkinter import ttk, simpledialog, messagebox
from PIL import Image, ImageTk
import datetime

class FaceRecognitionGUI:
    def __init__(self, window, window_title):
        self.window = window
        self.window.title(window_title)

        # Load the face recognizer and cascade classifier
        self.recognizer = cv2.face.LBPHFaceRecognizer_create()
        self.cascadePath = "haarcascade_frontalface_default.xml"
        self.faceCascade = cv2.CascadeClassifier(self.cascadePath)

        self.font = cv2.FONT_HERSHEY_SIMPLEX
        self.id = 0
        self.names = ['None', 'Luqman']

        # Create GUI elements
        self.canvas = tk.Canvas(window, width=640, height=480)
        self.canvas.pack()

        self.timestamp_label = ttk.Label(window, text="")
        self.timestamp_label.pack(pady=5)

        self.dataset_count_label = ttk.Label(window, text="Dataset count: 0")
        self.dataset_count_label.pack(pady=5)

        self.btn_start = ttk.Button(window, text="Start Recognition", command=self.start_recognition)
        self.btn_start.pack(pady=10)

        self.btn_stop = ttk.Button(window, text="Stop Recognition", command=self.stop_recognition)
        self.btn_stop.pack(pady=10)
        self.btn_stop.config(state='disabled')

        self.btn_create_dataset = ttk.Button(window, text="Create Dataset", command=self.create_dataset)
        self.btn_create_dataset.pack(pady=10)

        self.btn_train = ttk.Button(window, text="Train Model", command=self.train_model)
        self.btn_train.pack(pady=10)

        # Initialize video capture
        self.cam = cv2.VideoCapture(0)
        self.cam.set(3, 640)
        self.cam.set(4, 480)

        self.minW = 0.1*self.cam.get(3)
        self.minH = 0.1*self.cam.get(4)

        self.is_recognizing = False

        self.update_dataset_count()

        self.window.protocol("WM_DELETE_WINDOW", self.on_closing)
        self.window.mainloop()

    def start_recognition(self):
        self.is_recognizing = True
        self.btn_start.config(state='disabled')
        self.btn_stop.config(state='normal')
        self.btn_create_dataset.config(state='disabled')
        self.btn_train.config(state='disabled')
        
        # Load the trained model
        self.recognizer.read('trainer/trainer.yml')
        
        self.recognize_face()

    def stop_recognition(self):
        self.is_recognizing = False
        self.btn_start.config(state='normal')
        self.btn_stop.config(state='disabled')
        self.btn_create_dataset.config(state='normal')
        self.btn_train.config(state='normal')

    def recognize_face(self):
        if self.is_recognizing:
            ret, img = self.cam.read()
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            faces = self.faceCascade.detectMultiScale(
                gray,
                scaleFactor=1.2,
                minNeighbors=5,
                minSize=(int(self.minW), int(self.minH)),
            )

            for (x, y, w, h) in faces:
                cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
                id, confidence = self.recognizer.predict(gray[y:y+h, x:x+w])

                if confidence < 100:
                    id = self.names[id]
                    confidence = f"{round(100 - confidence)}%"
                else:
                    id = "unknown"
                    confidence = f"{round(100 - confidence)}%"

                cv2.putText(img, str(id), (x+5, y-5), self.font, 1, (255, 255, 255), 2)
                cv2.putText(img, str(confidence), (x+5, y+h-5), self.font, 1, (255, 255, 0), 1)

            # Add timestamp to the image
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            cv2.putText(img, timestamp, (10, 30), self.font, 0.7, (255, 255, 255), 1)

            # Update timestamp label
            self.timestamp_label.config(text=f"Current Time: {timestamp}")

            # Convert image to PhotoImage
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(img)
            img = ImageTk.PhotoImage(image=img)

            # Update canvas
            self.canvas.create_image(0, 0, anchor=tk.NW, image=img)
            self.canvas.image = img

            # Schedule next recognition
            self.window.after(10, self.recognize_face)

    def create_dataset(self):
        self.btn_start.config(state='disabled')
        self.btn_stop.config(state='disabled')
        self.btn_create_dataset.config(state='disabled')
        self.btn_train.config(state='disabled')

        face_id = simpledialog.askstring("Input", "Enter user id and press <return>")
        
        if face_id is None:
            self.btn_start.config(state='normal')
            self.btn_create_dataset.config(state='normal')
            self.btn_train.config(state='normal')
            return

        count = 0
        while True:
            ret, img = self.cam.read()
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = self.faceCascade.detectMultiScale(gray, 1.3, 5)

            for (x,y,w,h) in faces:
                cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,0), 2)     
                count += 1

                # Save the captured image into the datasets folder
                cv2.imwrite(f"dataset/User.{face_id}.{count}.jpg", gray[y:y+h,x:x+w])

            # Add count to the image
            cv2.putText(img, f"Count: {count}", (10, 30), self.font, 0.7, (0, 255, 0), 2)

            # Convert image to PhotoImage
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(img)
            img = ImageTk.PhotoImage(image=img)

            # Update canvas
            self.canvas.create_image(0, 0, anchor=tk.NW, image=img)
            self.canvas.image = img

            self.window.update()

            if count >= 300:  # Take 30 face samples and stop
                break

        print("\n [INFO] Exiting Program and cleanup stuff")
        self.btn_start.config(state='normal')
        self.btn_create_dataset.config(state='normal')
        self.btn_train.config(state='normal')
        self.update_dataset_count()

    def train_model(self):
        path = 'dataset'
        
        print ("\n [INFO] Training faces. It will take a few seconds. Wait ...")
        
        faces, ids = self.getImagesAndLabels(path)
        self.recognizer.train(faces, np.array(ids))

        # Save the model into trainer/trainer.yml
        if not os.path.exists('trainer'):
            os.makedirs('trainer')
        self.recognizer.write('trainer/trainer.yml')

        # Print the number of faces trained and end program
        print("\n [INFO] {0} faces trained. Exiting Program".format(len(np.unique(ids))))
        messagebox.showinfo("Training Complete", f"{len(np.unique(ids))} faces trained.")

    def getImagesAndLabels(self, path):
        imagePaths = [os.path.join(path,f) for f in os.listdir(path)]     
        faceSamples = []
        ids = []

        for imagePath in imagePaths:
            PIL_img = Image.open(imagePath).convert('L') # convert it to grayscale
            img_numpy = np.array(PIL_img,'uint8')

            id = int(os.path.split(imagePath)[-1].split(".")[1])
            faces = self.faceCascade.detectMultiScale(img_numpy)

            for (x,y,w,h) in faces:
                faceSamples.append(img_numpy[y:y+h,x:x+w])
                ids.append(id)

        return faceSamples, ids

    def update_dataset_count(self):
        path = 'dataset'
        if os.path.exists(path):
            count = len([name for name in os.listdir(path) if os.path.isfile(os.path.join(path, name))])
            self.dataset_count_label.config(text=f"Dataset count: {count}")
        else:
            self.dataset_count_label.config(text="Dataset count: 0")

    def on_closing(self):
        if self.cam.isOpened():
            self.cam.release()
        self.window.destroy()

# Create and run the GUI
root = tk.Tk()
FaceRecognitionGUI(root, "Face Recognition by Luqman")


 [INFO] Exiting Program and cleanup stuff

 [INFO] Training faces. It will take a few seconds. Wait ...

 [INFO] 1 faces trained. Exiting Program


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\Luqman\live_plot\.conda\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\Luqman\AppData\Local\Temp\ipykernel_31960\2454736968.py", line 71, in start_recognition
    self.recognize_face()
  File "C:\Users\Luqman\AppData\Local\Temp\ipykernel_31960\2454736968.py", line 94, in recognize_face
    id, confidence = self.recognizer.predict(gray[y:y+h, x:x+w])
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
cv2.error: Unknown C++ exception from OpenCV code
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\Luqman\live_plot\.conda\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\Luqman\AppData\Local\Temp\ipykernel_31960\2454736968.py", line 71, in start_recognition
    self.recognize_face()
  File "C:\Users\Luqman\AppData\Lo


 [INFO] Exiting Program and cleanup stuff

 [INFO] Training faces. It will take a few seconds. Wait ...

 [INFO] 1 faces trained. Exiting Program
